# CNN Custom untuk Klasifikasi Gambar

Notebook ini mencakup langkah-langkah:
1. Undersampling
2. Preprocessing
3. Split Dataset
4. Model CNN Custom
5. Training dan Evaluasi

In [ ]:
import os
import random
import shutil
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models


## Set Path dan Undersampling

In [ ]:
# Path ke dataset asli dan target output
original_dataset_dir = 'dataset'
base_dir = 'processed_dataset'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')
classes = os.listdir(original_dataset_dir)

# Fungsi undersampling dan split
def prepare_balanced_dataset(original_dir, target_dir, classes, train_split=0.7, val_split=0.2):
    if os.path.exists(target_dir):
        shutil.rmtree(target_dir)
    os.makedirs(target_dir)

    # Hitung jumlah minimum data
    min_count = min([len(os.listdir(os.path.join(original_dir, cls))) for cls in classes])

    for cls in classes:
        img_list = os.listdir(os.path.join(original_dir, cls))
        random.shuffle(img_list)
        img_list = img_list[:min_count]

        train_count = int(train_split * min_count)
        val_count = int(val_split * min_count)

        for split, count in zip(['train', 'val', 'test'], [train_count, val_count, min_count - train_count - val_count]):
            split_dir = os.path.join(target_dir, split, cls)
            os.makedirs(split_dir, exist_ok=True)
            for img in img_list[:count]:
                src = os.path.join(original_dir, cls, img)
                dst = os.path.join(split_dir, img)
                shutil.copy(src, dst)
            img_list = img_list[count:]

prepare_balanced_dataset(original_dataset_dir, base_dir, classes)


## Preprocessing dan Data Generator

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)


## Model CNN Custom

In [ ]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(classes), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    train_generator,
    epochs=15,
    validation_data=val_generator
)


## Evaluasi Model

In [ ]:
loss, acc = model.evaluate(test_generator)
print(f"Test Accuracy: {acc*100:.2f}%")

y_true = test_generator.classes
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)

print(classification_report(y_true, y_pred_classes, target_names=classes))
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred_classes))

plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Val')
plt.title('Accuracy')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Val')
plt.title('Loss')
plt.legend()
plt.show()
